In [ ]:
import numpy as np
from scipy.ndimage import uniform_filter1d
from scipy.signal import detrend


def find_peaks_original(x, scale=None, debug=False):
    """Find peaks in quasi-periodic noisy signals using AMPD algorithm.
    Automatic Multi-Scale Peak Detection originally proposed in
    "An Efficient Algorithm for Automatic Peak Detection in
    Noisy Periodic and Quasi-Periodic Signals", Algorithms 2012, 5, 588-603
    https://doi.org/10.1109/ICRERA.2016.7884365
    Optimized implementation by Igor Gotlibovych, 2018
    Parameters
    ----------
    x : ndarray
        1-D array on which to find peaks
    scale : int, optional
        specify maximum scale window size of (2 * scale + 1)
    debug : bool, optional
        if set to True, return the Local Scalogram Matrix, `LSM`,
        and scale with most local maxima, `l`,
        together with peak locations
    Returns
    -------
    pks: ndarray
        The ordered array of peak indices found in `x`
    """
    x = detrend(x)
    N = len(x)
    L = N // 2
    if scale:
        L = min(scale, L)

    # create LSM matix
    LSM = np.zeros((L, N), dtype=bool)
    for k in np.arange(1, L):
        LSM[k - 1, k:N - k] = (
            (x[0:N - 2 * k] < x[k:N - k]) & (x[k:N - k] > x[2 * k:N])
        )

    # Find scale with most maxima
    G = LSM.sum(axis=1)
    l_scale = np.argmax(G)

    # find peaks that persist on all scales up to l
    pks_logical = np.min(LSM[0:l_scale, :], axis=0)
    pks = np.flatnonzero(pks_logical)
    if debug:
        return pks, LSM, l_scale
    return pks


def find_peaks(x, scale=None, debug=False):
    """Find peaks in quasi-periodic noisy signals using AMPD algorithm.
    Extended implementation handles peaks near start/end of the signal.
    Optimized implementation by Igor Gotlibovych, 2018
    Parameters
    ----------
    x : ndarray
        1-D array on which to find peaks
    scale : int, optional
        specify maximum scale window size of (2 * scale + 1)
    debug : bool, optional
        if set to True, return the Local Scalogram Matrix, `LSM`,
        weigted number of maxima, 'G',
        and scale at which G is maximized, `l`,
        together with peak locations
    Returns
    -------
    pks: ndarray
        The ordered array of peak indices found in `x`
    """
    x = detrend(x)
    N = len(x)
    L = N // 2
    if scale:
        L = min(scale, L)

    # create LSM matix
    LSM = np.ones((L, N), dtype=bool)
    for k in np.arange(1, L + 1):
        LSM[k - 1, 0:N - k] &= (x[0:N - k] > x[k:N]
                                )  # compare to right neighbours
        LSM[k - 1, k:N] &= (x[k:N] > x[0:N - k])  # compare to left neighbours

    # Find scale with most maxima
    G = LSM.sum(axis=1)
    G = G * np.arange(
        N // 2, N // 2 - L, -1
    )  # normalize to adjust for new edge regions
    l_scale = np.argmax(G)

    # find peaks that persist on all scales up to l
    pks_logical = np.min(LSM[0:l_scale, :], axis=0)
    pks = np.flatnonzero(pks_logical)
    if debug:
        return pks, LSM, G, l_scale
    return pks


def find_peaks_adaptive(x, window=None, debug=False):
    """Find peaks in quasi-periodic noisy signals using ASS-AMPD algorithm.
    Adaptive Scale Selection Automatic Multi-Scale Peak Detection,
    an extension of AMPD -
    "An Efficient Algorithm for Automatic Peak Detection in
    Noisy Periodic and Quasi-Periodic Signals", Algorithms 2012, 5, 588-603
    https://doi.org/10.1109/ICRERA.2016.7884365
    Optimized implementation by Igor Gotlibovych, 2018
    Parameters
    ----------
    x : ndarray
        1-D array on which to find peaks
    window : int, optional
        sliding window size for adaptive scale selection
    debug : bool, optional
        if set to True, return the Local Scalogram Matrix, `LSM`,
        and `adaptive_scale`,
        together with peak locations
    Returns
    -------
    pks: ndarray
        The ordered array of peak indices found in `x`
    """
    x = detrend(x)
    N = len(x)
    if not window:
        window = N
    if window > N:
        window = N
    L = window // 2

    # create LSM matix
    LSM = np.ones((L, N), dtype=bool)
    for k in np.arange(1, L + 1):
        LSM[k - 1, 0:N - k] &= (x[0:N - k] > x[k:N]
                                )  # compare to right neighbours
        LSM[k - 1, k:N] &= (x[k:N] > x[0:N - k])  # compare to left neighbours

    # Create continuos adaptive LSM
    ass_LSM = uniform_filter1d(LSM * window, window, axis=1, mode='nearest')
    normalization = np.arange(L, 0, -1)  # scale normalization weight
    ass_LSM = ass_LSM * normalization.reshape(-1, 1)

    # Find adaptive scale at each point
    adaptive_scale = ass_LSM.argmax(axis=0)

    # construct reduced LSM
    LSM_reduced = LSM[:adaptive_scale.max(), :]
    mask = (np.indices(LSM_reduced.shape)[0] > adaptive_scale
            )  # these elements are outside scale of interest
    LSM_reduced[mask] = 1

    # find peaks that persist on all scales up to l
    pks_logical = np.min(LSM_reduced, axis=0)
    pks = np.flatnonzero(pks_logical)
    if debug:
        return pks, ass_LSM, adaptive_scale
    return pks

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.utils.data as Data
import torchvision.datasets
import torchvision.transforms
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import glob
# from pyampd.ampd import find_peaks


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

import matplotlib.pyplot as plt


import joblib

cpu


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/archive.zip

Archive:  gdrive/My Drive/archive.zip
  inflating: Samples/rec_1.csv       
  inflating: Samples/rec_10.csv      
  inflating: Samples/rec_100.csv     
  inflating: Samples/rec_101.csv     
  inflating: Samples/rec_102.csv     
  inflating: Samples/rec_103.csv     
  inflating: Samples/rec_104.csv     
  inflating: Samples/rec_105.csv     
  inflating: Samples/rec_106.csv     
  inflating: Samples/rec_107.csv     
  inflating: Samples/rec_108.csv     
  inflating: Samples/rec_109.csv     
  inflating: Samples/rec_11.csv      
  inflating: Samples/rec_110.csv     
  inflating: Samples/rec_111.csv     
  inflating: Samples/rec_112.csv     
  inflating: Samples/rec_113.csv     
  inflating: Samples/rec_114.csv     
  inflating: Samples/rec_115.csv     
  inflating: Samples/rec_116.csv     
  inflating: Samples/rec_117.csv     
  inflating: Samples/rec_118.csv     
  inflating: Samples/rec_119.csv     
  inflating: Samples/rec_12.csv      
  inflating: Samples/rec_120.csv     
  inflating:

In [ ]:
PPG_datas = []
ABP_datas = []
ECG_datas = []
i = 0
for name in glob.glob('./Samples/*.csv'):
    raw_training_data = pd.read_csv(name, header=None)
    raw_training_data = np.array(raw_training_data)
    #print(raw_training_data.shape)
    PPG_data = raw_training_data[0]
    ABP_data = raw_training_data[1]
    ECG_data = raw_training_data[2]

    PPG_datas.append(PPG_data)
    ABP_datas.append(ABP_data)
    ECG_datas.append(ECG_data)
    # i = i + 1
 #   print(i)
#     if (i == 100):
#         break

PPG_datas = np.array(PPG_datas)
ABP_datas = np.array(ABP_datas)
ECG_datas = np.array(ECG_datas)
# raw_training_data.head()

<ipython-input-6-cafb0a523fd7>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  PPG_datas = np.array(PPG_datas)
<ipython-input-6-cafb0a523fd7>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ABP_datas = np.array(ABP_datas)
<ipython-input-6-cafb0a523fd7>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ECG_datas = np.array(ECG_da

In [ ]:
import pywt
from sklearn.decomposition import PCA

def generate_wavelet_vector(X, wavelet_family, decomposition_level):
    coefficients = pywt.wavedec(X, wavelet_family, level=decomposition_level)
    vector = np.array([])
    for coeffs in coefficients:
        vector = np.concatenate((vector, coeffs))
    return vector

In [ ]:
SAMPLE_FREQ = 125

In [ ]:
wavelet_family = 'db4'
decomposition_level = 5

wavelet_vectors = []
SBP_data = []
DBP_data = []
MAP_data = []

for j in range(len(PPG_datas)):
    sec_15 = 15 * SAMPLE_FREQ
    PPG_data = PPG_datas[j]
    ABP_data = ABP_datas[j]
    PPG_peaks = find_peaks(PPG_data, scale=SAMPLE_FREQ)
    for i in range(2, PPG_peaks.shape[0]):
        X = PPG_data[PPG_peaks[i - 1]:PPG_peaks[i]]
        if len(X) < SAMPLE_FREQ:
            wavelet_vector = generate_wavelet_vector(X, wavelet_family, decomposition_level)

            SBP = np.max(ABP_data[PPG_peaks[i - 1]:PPG_peaks[i - 1] + sec_15])
            DBP = np.min(ABP_data[PPG_peaks[i - 1]:PPG_peaks[i - 1] + sec_15])
            MAP = SBP / 3 + 2 * DBP / 3

            wavelet_vectors.append(wavelet_vector)
            SBP_data.append(SBP)
            DBP_data.append(DBP)
            MAP_data.append(MAP)

/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 5 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [ ]:
wavelet_vectors = np.array(wavelet_vectors)
SBP_data = np.array(SBP_data)
DBP_data = np.array(DBP_data)
MAP_data = np.array(MAP_data)

print(wavelet_vectors.shape)
print(SBP_data.shape)

# Find the maximum length of the wavelet vectors
max_length = max([len(vector) for vector in wavelet_vectors])

# Pad the wavelet vectors with zeros to make them uniform length
padded_wavelet_vectors = []
for vector in wavelet_vectors:
    padded_vector = np.pad(vector, (0, max_length - len(vector)), mode='constant')
    padded_wavelet_vectors.append(padded_vector)

padded_wavelet_vectors = np.array(padded_wavelet_vectors)

<ipython-input-10-6f3e639507bb>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  wavelet_vectors = np.array(wavelet_vectors)


(196625,)
(196625,)


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=43)
pca_wavelet_vectors = pca.fit_transform(padded_wavelet_vectors)

print(pca_wavelet_vectors.shape)


(196625, 43)


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, SBP_train, SBP_test, DBP_train, DBP_test, MAP_train, MAP_test = train_test_split(
    pca_wavelet_vectors, SBP_data, DBP_data, MAP_data, test_size=0.1, random_state=42)

In [ ]:
def AAMI_standard(predict, test):
    total = len(predict)
    ME = np.mean(predict - test)
    MAE = np.mean(np.abs(predict-test))
    SD = np.std(predict-test)

    return total, ME, MAE, SD

def BHS_standard(predict, test):
    total = len(predict)
    mm5 = np.sum(np.abs(predict-test)<=5)
    mm10 = np.sum(np.abs(predict-test)<=10)
    mm15 = np.sum(np.abs(predict-test)<=15)
    return total, mm5, mm10, mm15

In [ ]:
EPOCHS = 2
LR = 0.001

In [ ]:
class ANN(nn.Module):
    def __init__(self, input_size):
        super(ANN, self).__init__()
        self.input = nn.Linear(input_size, 1024)
        self.hidden1 = nn.Linear(1024, 1024)
        self.hidden2 = nn.Linear(1024, 1024)
        self.output = nn.Linear(1024, 1)
        self.batch_norm1 = nn.BatchNorm1d(1024)
        self.batch_norm2 = nn.BatchNorm1d(1024)
        self.batch_norm3 = nn.BatchNorm1d(1024)
        self.dropout = nn.Dropout(p=0.2)
        self.activation = nn.ReLU()

    def forward(self, x):
        out = self.input(x)
        out = self.batch_norm1(out)
        out = self.activation(out)
        out = self.dropout(out)

        out = self.hidden1(out)
        out = self.batch_norm2(out)
        out = self.activation(out)
        out = self.dropout(out)

        out = self.hidden2(out)
        out = self.batch_norm3(out)
        out = self.activation(out)
        out = self.dropout(out)

        out = self.output(out)
        return out



In [ ]:
torch.set_default_tensor_type(torch.FloatTensor)

class ANNModule():
    def __init__(self, net, train_loader, test_loader, EPOCH=20, LR=0.0001):
        self.net = net
        self.optimizer = torch.optim.Adam(net.parameters(), lr = LR)
        self.criterion = nn.MSELoss()
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.EPOCHS = EPOCH
        self.LR = LR

        self.net = self.net.to(device)
        if device == 'cuda':
            self.net = torch.nn.DataParallel(self.net)
            torch.backends.cudnn.benchmark = True

    def start(self):

        self.history_train_loss = []
        self.history_test_loss = []
#         history_train_accuracy = []
#         history_test_accuracy = []
        for epoch in range(self.EPOCHS):
            print('Epoch:', epoch)
            train_loss = self.train()
            test_loss = self.test()

            self.history_train_loss.append(train_loss)
            self.history_test_loss.append(test_loss)
#             history_train_accuracy.append(train_accuracy)
#             history_test_accuracy.append(test_accuracy)

    def train(self):
        self.net.train()
        train_loss = 0
        correct = 0
        total = 0
        for step, (batch_X, batch_y) in enumerate(self.train_loader):
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            self.optimizer.zero_grad()
            outputs = self.net(batch_X).double()

            loss = self.criterion(outputs.reshape(-1), batch_y)
            loss.backward()
            self.optimizer.step()

            train_loss += loss.item()
            #_, predicted = outputs.max(1)
            #total += batch_y.size(0)
            #correct += predicted.eq(batch_y).sum().item()

        print('【Training】Loss: %.3f ' % ( train_loss))
        return train_loss

    def test(self):
        self.net.eval()

        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for step, (batch_X, batch_y) in enumerate(self.test_loader):
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = self.net(batch_X).double()
                loss = self.criterion(outputs.reshape(-1), batch_y)

                test_loss += loss.item()
#                 _, predicted = outputs.max(1)
#                 #print(predicted, batch_y)
#                 total += batch_y.size(0)
#                 correct += predicted.eq(batch_y).sum().item()

        print('【Testing】Loss: %.3f )' % ( test_loss))
        return test_loss

    def predict(self, test_loader):
        outputs = []
        self.net.eval()

        with torch.no_grad():
            for step, (batch_X, batch_y) in enumerate(test_loader):
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                output = self.net(batch_X).double().cpu().numpy().reshape(-1)
                for i in range(len(output)):
                    outputs.append(output[i])

        return outputs

    def get_model(self):
        return self.net.state_dict()

In [ ]:
from sklearn.model_selection import train_test_split

L = 43

x_train, x_test, SBP_train, SBP_test, DBP_train, DBP_test, MAP_train, MAP_test = train_test_split(
                                            pca_wavelet_vectors, SBP_data, DBP_data, MAP_data, test_size=0.1, random_state=42)


x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
x_test = torch.from_numpy(x_test).type(torch.FloatTensor)
SBP_train = torch.from_numpy(SBP_train)
SBP_test = torch.from_numpy(SBP_test)
DBP_train = torch.from_numpy(DBP_train)
DBP_test = torch.from_numpy(DBP_test)
MAP_train = torch.from_numpy(MAP_train)
MAP_test = torch.from_numpy(MAP_test)

print(x_train.shape, x_test.shape, SBP_train.shape, SBP_test.shape)

SBP_train_dataset = Data.TensorDataset(x_train, SBP_train)
SBP_test_dataset = Data.TensorDataset(x_test, SBP_test)

SBP_train_loader = Data.DataLoader(
    dataset=SBP_train_dataset,
    batch_size=128,
)

SBP_test_loader = Data.DataLoader(
    dataset=SBP_test_dataset,
    batch_size=10,
)

DBP_train_dataset = Data.TensorDataset(x_train, DBP_train)
DBP_test_dataset = Data.TensorDataset(x_test, DBP_test)

DBP_train_loader = Data.DataLoader(
    dataset=DBP_train_dataset,
    batch_size=128,
)

DBP_test_loader = Data.DataLoader(
    dataset=DBP_test_dataset,
    batch_size=10,
)

MAP_train_dataset = Data.TensorDataset(x_train, MAP_train)
MAP_test_dataset = Data.TensorDataset(x_test, MAP_test)

MAP_train_loader = Data.DataLoader(
    dataset=MAP_train_dataset,
    batch_size=128,
)

MAP_test_loader = Data.DataLoader(
    dataset=MAP_test_dataset,
    batch_size=10,
)


torch.Size([176962, 43]) torch.Size([19663, 43]) torch.Size([176962]) torch.Size([19663])


In [ ]:
SBP_ann_module = ANNModule(ANN(input_size=L), SBP_train_loader, SBP_test_loader, EPOCH=200, LR=LR)
SBP_ann_module.start()

Epoch: 0
【Training】Loss: 2208646.195 
【Testing】Loss: 516690.737 )
Epoch: 1
【Training】Loss: 382960.898 
【Testing】Loss: 462076.544 )
Epoch: 2
【Training】Loss: 352671.321 
【Testing】Loss: 425015.012 )
Epoch: 3
【Training】Loss: 332808.794 
【Testing】Loss: 407127.122 )
Epoch: 4
【Training】Loss: 318978.871 
【Testing】Loss: 392323.805 )
Epoch: 5
【Training】Loss: 307070.080 
【Testing】Loss: 379309.210 )
Epoch: 6
【Training】Loss: 298116.867 
【Testing】Loss: 365663.206 )
Epoch: 7
【Training】Loss: 290489.393 
【Testing】Loss: 361374.987 )
Epoch: 8
【Training】Loss: 283634.575 
【Testing】Loss: 352393.496 )
Epoch: 9
【Training】Loss: 277086.526 
【Testing】Loss: 342711.935 )
Epoch: 10
【Training】Loss: 270443.836 
【Testing】Loss: 342080.798 )
Epoch: 11
【Training】Loss: 264362.281 
【Testing】Loss: 334605.242 )
Epoch: 12
【Training】Loss: 260691.129 
【Testing】Loss: 328051.314 )
Epoch: 13
【Training】Loss: 256343.806 
【Testing】Loss: 324540.325 )
Epoch: 14
【Training】Loss: 251824.192 
【Testing】Loss: 321017.363 )
Epoch: 15
【Training

In [ ]:
DBP_ann_module = ANNModule(ANN(input_size=L), DBP_train_loader, DBP_test_loader, EPOCH=200, LR=LR)
DBP_ann_module.start()

In [ ]:
MAP_ann_module = ANNModule(ANN(input_size=L), MAP_train_loader, MAP_test_loader, EPOCH=200, LR=LR)
MAP_ann_module.start()

In [ ]:
# temp_predict = []
# for predicts in SBP_predict:
#     for i in range(len(predicts))
#         temp_predict.append(predicts[i])
# print(SBP_predict[0:20])

# print(SBP_test[0:20])
import time

# Record the start time

start_time = time.time()

SBP_predict = np.array(SBP_ann_module.predict(SBP_test_loader)).reshape(-1)
SBP_test = np.array(SBP_test)
DBP_predict = np.array(DBP_ann_module.predict(DBP_test_loader)).reshape(-1)
DBP_test = np.array(DBP_test)
MAP_predict = np.array(MAP_ann_module.predict(MAP_test_loader)).reshape(-1)
MAP_test = np.array(MAP_test)

print('Artificial Nueral Network')

total, ME, MAE, SD = AAMI_standard(SBP_predict, SBP_test)
print()
print()
print('----------------SBP-----------------')
print()
print('-------------AAMI standard----------')
print('ME             MAE             SD           total')
print('%.3f         %.3f         %.3f          %d' % (ME,MAE,SD,total))
#print('ME: ', ME, '   MAE: ', MAE, '   SD: ', SD, '   total: ', total)
total, mm5, mm10, mm15 = BHS_standard(SBP_predict, SBP_test)
print()
print('-------------BHS standard------------')
print('<5mmHg        <10mmHg        <15mmHg        total')
print('%d            %d           %d          %d' % (mm5,mm10,mm15,total))
print('%.3f%s        %.3f%s       %.3f%s         %d' % (mm5/total*100, '%',mm10/total*100, '%',mm15/total*100, '%', total))

total, ME, MAE, SD = AAMI_standard(DBP_predict, DBP_test)
print()
print()
print('----------------DBP-----------------')
print()
print('-------------AAMI standard----------')
print('ME             MAE             SD           total')
print('%.3f         %.3f         %.3f          %d' % (ME,MAE,SD,total))
#print('ME: ', ME, '   MAE: ', MAE, '   SD: ', SD, '   total: ', total)
total, mm5, mm10, mm15 = BHS_standard(DBP_predict, DBP_test)
print()
print('-------------BHS standard------------')
print('<5mmHg        <10mmHg        <15mmHg        total')
print('%d            %d           %d          %d' % (mm5,mm10,mm15,total))
print('%.3f%s        %.3f%s       %.3f%s         %d' % (mm5/total*100, '%',mm10/total*100, '%',mm15/total*100, '%', total))

total, ME, MAE, SD = AAMI_standard(MAP_predict, MAP_test)
print()
print()
print('----------------MAP-----------------')
print()
print('-------------AAMI standard----------')
print('ME             MAE             SD           total')
print('%.3f         %.3f         %.3f          %d' % (ME,MAE,SD,total))
#print('ME: ', ME, '   MAE: ', MAE, '   SD: ', SD, '   total: ', total)
total, mm5, mm10, mm15 = BHS_standard(MAP_predict, MAP_test)
print()
print('-------------BHS standard------------')
print('<5mmHg        <10mmHg        <15mmHg        total')
print('%d            %d           %d          %d' % (mm5,mm10,mm15,total))
print('%.3f%s        %.3f%s       %.3f%s         %d' % (mm5/total*100, '%',mm10/total*100, '%',mm15/total*100, '%', total))

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print("Time taken:", elapsed_time, "seconds")

In [ ]:
print('Artificial Neural Network')

SBP_cc= np.corrcoef(SBP_predict, SBP_test)
DBP_cc= np.corrcoef(DBP_predict, DBP_test)
MAP_cc= np.corrcoef(MAP_predict, MAP_test)
print()
print()
print('------------Correlation Coefficient-------------')
print()
print('SBP: %.3f' % (SBP_cc[0, 1]))
print()
print('DBP: %.3f' % (DBP_cc[0, 1]))
print()
print('MAP: %.3f' % (MAP_cc[0, 1]))

In [ ]:
# confusion matrix

import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate confusion matrix
cm = confusion_matrix(SBP_test, SBP_predict)

# Display confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')

# Set labels, title, and axis ticks
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix")
plt.show()

